## Getting the data

In [2]:
import tensorflow_datasets as tf_ds
import tensorflow as tf
import numpy as np

d_train , d_vs , d_test = tf_ds.load(
    "cats_vs_dogs" ,
    split = ["train[0:40%]" , "train[40%:50%]" , "train[50%:60%]"], #10% for validation and 10% for test
    as_supervised=True, #include labels
    )
print(tf.data.experimental.cardinality(d_train))
print(tf.data.experimental.cardinality(d_vs))
print(tf.data.experimental.cardinality(d_test))

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteSSH24D/cats_vs_dogs-train.tfrecord



Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.
tf.Tensor(9305, shape=(), dtype=int64)
tf.Tensor(2326, shape=(), dtype=int64)
tf.Tensor(2326, shape=(), dtype=int64)


## Standardizing the data

In [0]:
# we need to set a fix size for our input images as they may come in various sizes
input_shape = (150,150)

d_train = d_train.map(lambda x,y : (tf.image.resize(x,input_shape) , y)) #x being our image and y being it's label
d_vs = d_vs.map(lambda x,y :(tf.image.resize(x,input_shape),y))
d_test = d_test.map(lambda x,y : (tf.image.resize(x,input_shape),y))

In [0]:
#let's batch the data and use caching and prefetching to optimise loading speed
batch_size = 32
d_train = d_train.cache().batch(batch_size).prefetch(buffer_size = 10)
d_vs = d_vs.cache().batch(batch_size).prefetch(buffer_size = 10)
d_test = d_test.cache().batch(batch_size).prefetch(buffer_size = 10)

#data augmentation

when we dont have alot of data we increase data points by data augmentation

In [0]:
from tensorflow import keras
from tensorflow.keras import layers

data_aug = keras.Sequential(
    [
      layers.experimental.preprocessing.RandomFlip("horizontal"),
      layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

#Building the model

In [17]:
base_model = keras.applications.Xception(
      #pretrained weights of xception model on imagenet dataset
      weights="imagenet" ,
      input_shape = (150,150,3) , #3 cause of rgb
      include_top = False , #removing / not including the classifier of xception as we will need to use our own
)

#freezing the base model
base_model.trainable = False

#creating our new model to be placed on top
inputs = keras.Input(shape=(150,150,3))

#applying data augmentation
x = data_aug(inputs)

#pretrained xception weights required normalised that is to be transformed from (0,255) to (-1,1)
#the normalise layer does the following : (input-mean)/sqrt(var)

#creating the norm layer
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5]*3)
var = mean**2

# getting scaled inputs
x = norm_layer(x)
norm_layer.set_weights([mean,var])

#the base model has batch normalization layers so they must be kept in inference mode when we unfreeze the base model for fine tunning
#we will run the base model in inference model here
x = base_model(x,training=False) 
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32,activation="relu")(x)
x = keras.layers.Dropout(0.5)(x)
final_output = keras.layers.Dense(1,activation="sigmoid")(x)
new_model = keras.Model(inputs = inputs , outputs = final_output)

new_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 150, 150, 3)       0         
_________________________________________________________________
normalization_2 (Normalizati (None, 150, 150, 3)       7         
_________________________________________________________________
xception (Model)             (None, 5, 5, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                6556

#Train the top layer

In [18]:
new_model.compile(
    optimizer = keras.optimizers.Adam() ,
    loss = keras.losses.BinaryCrossentropy(from_logits=True) ,
    metrics = [keras.metrics.BinaryAccuracy()] ,
) 
epoch =20

new_model.fit(d_train,epochs = epoch , validation_data= d_vs)

Epoch 1/20
291/291 [==============================] - 28s 96ms/step - loss: 0.5471 - binary_accuracy: 0.9085 - val_loss: 0.5289 - val_binary_accuracy: 0.9592
Epoch 2/20
291/291 [==============================] - 28s 95ms/step - loss: 0.5350 - binary_accuracy: 0.9286 - val_loss: 0.5211 - val_binary_accuracy: 0.9699
Epoch 3/20
291/291 [==============================] - 27s 94ms/step - loss: 0.5329 - binary_accuracy: 0.9344 - val_loss: 0.5264 - val_binary_accuracy: 0.9639
Epoch 4/20
291/291 [==============================] - 27s 94ms/step - loss: 0.5308 - binary_accuracy: 0.9396 - val_loss: 0.5262 - val_binary_accuracy: 0.9639
Epoch 5/20
291/291 [==============================] - 28s 95ms/step - loss: 0.5298 - binary_accuracy: 0.9385 - val_loss: 0.5195 - val_binary_accuracy: 0.9695
Epoch 6/20
291/291 [==============================] - 27s 94ms/step - loss: 0.5294 - binary_accuracy: 0.9396 - val_loss: 0.5232 - val_binary_accuracy: 0.9682
Epoch 7/20
291/291 [==============================] 

#fine tunning the entire model

In [19]:
#here we unfreeze the base model and train the entire model end to end with a low learning rate
# since we passed training=False when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
new_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 150, 150, 3)       0         
_________________________________________________________________
normalization_2 (Normalizati (None, 150, 150, 3)       7         
_________________________________________________________________
xception (Model)             (None, 5, 5, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                6556

In [20]:
new_model.compile(
    optimizer = keras.optimizers.Adam(1e-5) , #keeping learing rate small so that weights dont change drastically
    loss = keras.losses.BinaryCrossentropy(from_logits=True),
    metrics = [keras.metrics.BinaryAccuracy()],
)
epochs = 15
new_model.fit(d_train , epochs = epochs , validation_data= d_vs)

Epoch 1/15
291/291 [==============================] - 108s 371ms/step - loss: 0.5243 - binary_accuracy: 0.9528 - val_loss: 0.5162 - val_binary_accuracy: 0.9768
Epoch 2/15
291/291 [==============================] - 109s 373ms/step - loss: 0.5200 - binary_accuracy: 0.9617 - val_loss: 0.5161 - val_binary_accuracy: 0.9721
Epoch 3/15
291/291 [==============================] - 108s 370ms/step - loss: 0.5195 - binary_accuracy: 0.9608 - val_loss: 0.5161 - val_binary_accuracy: 0.9785
Epoch 4/15
291/291 [==============================] - 107s 369ms/step - loss: 0.5164 - binary_accuracy: 0.9690 - val_loss: 0.5145 - val_binary_accuracy: 0.9768
Epoch 5/15
291/291 [==============================] - 108s 371ms/step - loss: 0.5154 - binary_accuracy: 0.9724 - val_loss: 0.5142 - val_binary_accuracy: 0.9811
Epoch 6/15
291/291 [==============================] - 108s 371ms/step - loss: 0.5172 - binary_accuracy: 0.9687 - val_loss: 0.5151 - val_binary_accuracy: 0.9802
Epoch 7/15
291/291 [====================

we saw increase in accuracy and a lower log-loss by fine tunning the complete model